# Processamento de Imagem e Visão
## Trabalho Prático 1
### Alunos: Belarmino Sacate (52057) e Miguel Ferreira (51878)

------
# 1. INTRODUÇÃO

-------------
## BIBLIOTECAS

In [119]:
import cv2
import numpy as np

------------
## 3.VARIAVEIS

In [134]:
ficheiro_video = "Vídeo - Enunciado A.avi"
area_minima = 350
distancia_limite = 80
frames_fundo = 40
fps = 25
pixels_por_metro = 10

pts1 = [[24, 217], 
        [78, 237], 
        [114, 82], 
        [99, 79]
       ]

pts2 = [[83, 235], 
        [145, 238], 
        [134, 84], 
        [118, 84]
       ]

pts3 = [[152, 237], 
        [138, 82], 
        [155, 79], 
        [218, 232]
       ]

roi1 = np.array(pts1, np.int32)
roi2 = np.array(pts2, np.int32)
roi3 = np.array(pts3, np.int32)

rois = {1: roi1,
        2: roi2,
        3: roi3
        }

faixa1 = []
faixa2 = []
faixa3 = []

------------
## METODOS

In [135]:
def obter_fundo(caminho_video, num_frames=40):
    
    frames = []
    video = cv2.VideoCapture(caminho_video)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    random_frames = np.random.randint(0, total_frames, num_frames)

    
    for indice in random_frames:
        video.set(cv2.CAP_PROP_POS_FRAMES, indice)
        _, frame = video.read()
        frames.append(frame) 
            
    video.release()
    

    if frames == []:
        print("Falha ao ler frames para o modelo de fundo")
        
    fundo = np.median(frames, axis=0).astype(dtype=np.uint8)

    print("Fundo gerado")
    return fundo

In [136]:
def detetar_movimento(frame_cinza, fundo_cinza):
    diferenca = cv2.absdiff(frame_cinza, fundo_cinza)
    _, binario = cv2.threshold(diferenca, 30, 255, cv2.THRESH_BINARY)
    return binario

In [137]:
def operadores_morfologicos(mascara_binaria, roi):
    
    kernel = np.ones((5, 5), np.uint8)

    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    mascara = cv2.morphologyEx(mascara_binaria, cv2.MORPH_CLOSE, kernel)
    mascara = cv2.dilate(mascara, kernel1)

    
    mascara_roi = np.zeros_like(mascara)
    cv2.fillPoly(mascara_roi, [roi], 255)

    mascara_processada = cv2.bitwise_and(mascara, mascara_roi)
    
    return mascara_processada

In [138]:
def detetar_veiculos(mascara_processada):
    
    contornos, _ = cv2.findContours(mascara_processada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detecoes = []
    
    for contorno in contornos:
        area = cv2.contourArea(contorno)
 
        if area > area_minima:
            x, y, largura, altura = cv2.boundingRect(contorno)
            centro_x, centro_y = x + largura // 2, y + altura // 2
            detecoes.append((centro_x, centro_y, x, y, largura, altura))

    return detecoes

In [139]:
def calcular_velocidade(centro_anterior, centro_atual):
    pos_x_anterior, pos_y_anterior = centro_anterior
    centro_x_atual, centro_y_atual = centro_atual
    distancia = np.hypot(centro_x_atual - pos_x_anterior, centro_y_atual - pos_y_anterior)
    if distancia < 1:
        return None
    
    distancia_metros = distancia / pixels_por_metro
    velocidade = (distancia_metros * fps) * 3.6
    if velocidade < 0 or velocidade > 200:
        return None
    
    return velocidade

In [140]:
def atualizar_tracking(deteccoes, veiculos, proximo_id_veiculo):
    veiculos_ocupados = []
    novos_veiculos = 0
    
    for centro_x, centro_y, x, y, largura, altura, numero_faixa in deteccoes:
        id_melhor_match = None
        distancia_melhor_match = 99999
        
        for id_veiculo, dados_veiculo in veiculos.items():
            pos_x_anterior, pos_y_anterior = dados_veiculo["centro"]
            distancia = np.hypot(centro_x - pos_x_anterior, centro_y - pos_y_anterior)
            mesma_faixa = abs(dados_veiculo["faixa"] - numero_faixa) <= 1
            
            if distancia < distancia_melhor_match and distancia < distancia_limite and mesma_faixa:
                distancia_melhor_match = distancia
                id_melhor_match = id_veiculo
        
        if id_melhor_match:
            if id_melhor_match not in veiculos_ocupados:
                veiculos_ocupados.append(id_melhor_match)
            dados_veiculo = veiculos[id_melhor_match]
            
            velocidade_calculada = calcular_velocidade(dados_veiculo["centro"], (centro_x, centro_y))
            if velocidade_calculada is not None:
                dados_veiculo["velocidade"] = velocidade_calculada
                
                if numero_faixa == 1:
                    faixa1.append(velocidade_calculada)
                elif numero_faixa == 2:
                    faixa2.append(velocidade_calculada)
                else:
                    faixa3.append(velocidade_calculada)
            
            dados_veiculo["centro"] = (centro_x, centro_y)
            dados_veiculo["bbox"] = (x, y, largura, altura)
            dados_veiculo["faixa"] = numero_faixa
            dados_veiculo["perdidos"] = 0
        else:
            proximo_id_veiculo += 1
            novos_veiculos += 1
            veiculos[proximo_id_veiculo] = {
                "centro": (centro_x, centro_y),
                "bbox": (x, y, largura, altura),
                "faixa": numero_faixa,
                "velocidade": 0,
                "perdidos": 0
            }
    
    veiculos_a_remover = []
    for id_veiculo, dados_veiculo in veiculos.items():
        if id_veiculo not in veiculos_ocupados:
            dados_veiculo["perdidos"] += 1
            if dados_veiculo["perdidos"] > 10:
                veiculos_a_remover.append(id_veiculo)
    
    for id_veiculo in veiculos_a_remover:
        del veiculos[id_veiculo]
    
    return veiculos, proximo_id_veiculo, novos_veiculos

In [141]:
def desenhar_rois(frame):
    cv2.polylines(frame, [roi1], True, (0, 255, 0), 2)
    cv2.polylines(frame, [roi2], True, (0, 255, 0), 2)
    cv2.polylines(frame, [roi3], True, (0, 255, 0), 2)

In [142]:
def desenhar_veiculos(frame, veiculos):
    for id_veiculo, dados_veiculo in veiculos.items():
        x, y, largura, altura = dados_veiculo["bbox"]
        velocidade = int(dados_veiculo["velocidade"])
        cv2.rectangle(frame, (x, y), (x + largura, y + altura), (0, 0, 255), 1)
        cv2.putText(frame, f"ID {id_veiculo} {velocidade} km/h", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 0), 1)

In [146]:
ef desenhar_estatisticas(frame, contador_total_veiculos):
    velocidade_media_faixa1 = np.mean(faixa1) if faixa1 else 0
    velocidade_media_faixa2 = np.mean(faixa2) if faixa2 else 0
    velocidade_media_faixa3 = np.mean(faixa3) if faixa3 else 0
    
    cv2.putText(frame, f"VM1: {velocidade_media_faixa1:.1f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)
    cv2.putText(frame, f"VM2: {velocidade_media_faixa2:.1f}", (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)
    cv2.putText(frame, f"VM3: {velocidade_media_faixa3:.1f}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)
    cv2.putText(frame, f"TOTAL: {contador_total_veiculos}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

SyntaxError: invalid syntax (1841903283.py, line 1)

In [145]:
fundo = obter_fundo(ficheiro_video, frames_fundo)
fundo_gray = cv2.cvtColor(fundo, cv2.COLOR_BGR2GRAY)
cap = cv2.VideoCapture(ficheiro_video)
veiculos = {}
proximo_id_veiculo = 0
contador_total_veiculos = 0

while True:
    sucesso, frame = cap.read()
    if not sucesso:
        break
    
    frame_cinza = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_visual = frame.copy()
    mascara_total = np.zeros_like(frame_cinza)
    deteccoes = []
    
    #--------------FAIXA1----------------#
    mascara_faixa1 = detetar_movimento(frame_cinza, fundo_gray)
    mascara_faixa1 = operadores_morfologicos(mascara_faixa1, rois[1])
    mascara_total |= mascara_faixa1
    deteccoes_faixa1 = detetar_veiculos(mascara_faixa1)
    for centro_x, centro_y, x, y, largura, altura in deteccoes_faixa1:
        deteccoes.append((centro_x, centro_y, x, y, largura, altura, 1))
    
    #--------------FAIXA2----------------#
    mascara_faixa2 = detetar_movimento(frame_cinza, fundo_gray)
    mascara_faixa2 = operadores_morfologicos(mascara_faixa2, rois[2])
    mascara_total |= mascara_faixa2
    deteccoes_faixa2 = detetar_veiculos(mascara_faixa2)
    for centro_x, centro_y, x, y, largura, altura in deteccoes_faixa2:
        deteccoes.append((centro_x, centro_y, x, y, largura, altura, 2))
    
    #--------------FAIXA3----------------#
    mascara_faixa3 = detetar_movimento(frame_cinza, fundo_gray)
    mascara_faixa3 = operadores_morfologicos(mascara_faixa3, rois[3])
    mascara_total |= mascara_faixa3
    deteccoes_faixa3 = detetar_veiculos(mascara_faixa3)
    for centro_x, centro_y, x, y, largura, altura in deteccoes_faixa3:
        deteccoes.append((centro_x, centro_y, x, y, largura, altura, 3))

    
    veiculos, proximo_id_veiculo, novos_veiculos = atualizar_tracking(deteccoes, veiculos, proximo_id_veiculo)
    contador_total_veiculos += novos_veiculos
    
    desenhar_rois(frame_visual)
    desenhar_veiculos(frame_visual, veiculos)
    desenhar_estatisticas(frame_visual, contador_total_veiculos)
    
    cv2.imshow("Contagem de Veículos", frame_visual)
    cv2.imshow("Máscara", mascara_total)
    
    tecla = cv2.waitKey(20)
    if tecla == 27:
        break

cap.release()
cv2.destroyAllWindows()
print("total de veiculos detectados: ", contador_total_veiculos)

Fundo gerado
total de veiculos detectados:  11


------------
# 5. CONCLUSÃO